<a href="https://colab.research.google.com/github/SerasLain/catandthekittens/blob/master/CAT_collocates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Поиск коллокаций в CAT

А вообще-то код должен быть там https://github.com/MariaFjodorowa/catandthekittens/tree/develop/collocations

In [0]:
# Connection to my Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Some useful paths
cat_d = '/content/gdrive/My Drive/CAT/'
cat_together = cat_d + 'Склеенные/'
cat_cleaned = cat_d + 'Cleaned/'
cat_conllu = '/content/gdrive/My Drive/Новые conll по доменам/NewVers/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Препроцессинг нового корпуса

In [0]:
import os
import re
import nltk
from nltk.corpus import stopwords


In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# Connection to my Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Файл с целиковым корпусом проще записать из остальных файлов, чем препроцессить отдельно
cat_d = '/content/gdrive/My Drive/CAT/'
cat_together = cat_d + 'Склеенные/'
files = os.listdir(cat_together)
files.pop(0)


'CAT.txt'

In [0]:
#From https://github.com/MariaFjodorowa/catandthekittens/blob/develop/CYBERsuffering/Preprocessing%20and%20modelling%20example.ipynb
# by Alex Klimov
#относительно хороший результат чистки корпуса
stopwords = ['б', 'г', 'д', 'е', 'ё', 'ж', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'гг']
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
goal_d = cat_d + 'Cleaned/'
if not os.path.exists(goal_d):
  os.mkdir(goal_d)
work_d = cat_together
for i, file in enumerate(files):
    text = os.path.abspath(cat_together + file)
    with open(text, 'r', encoding = 'utf-8') as t:
      txt = t.read()  
    txt = txt.lower()
    tokenized_text = tokenizer.tokenize(txt)
    lst = []
    for tryingtoclean in tokenized_text:
        tryingtoclean = re.sub('(данные об авторе)(\.)?.*', '', tryingtoclean, flags=re.S)
        tryingtoclean = re.sub(r'[a-zA-z]', '', tryingtoclean)
        tryingtoclean = re.sub(r'_', '', tryingtoclean)
        tryingtoclean = re.sub(r'\d', '', tryingtoclean)
        tryingtoclean = re.sub(r'[ñâÿçàíšßãóìøþôõýæùúñî¹áîřðûɫɥɨêɜǎäiöüéɨïіïëèåòè]', '', tryingtoclean)
        words = re.findall(r'\w+', tryingtoclean)
        clean = filter(lambda a: a not in stopwords, words)
        x = ' '.join(clean).rstrip()
        lst.append(x)
    xxx = '. '.join(lst).rstrip() + '.'
    dotpattern = re.compile(r'(\. \.)+')
    xxx = re.sub(dotpattern, '', xxx)
    spacepattern = re.compile(r'(  )+')
    xxx = re.sub(spacepattern, '', xxx)
    dotpattern2 = re.compile(r'( \.)+')
    xxx = re.sub(dotpattern2, '', xxx)
# Отсюда убран небольшой кусок кода, поскольку он ошибочно срабатывал на строках типа "однако он", превращая их в "однакн"
    goal_f = open(goal_d + file, 'w', encoding = 'utf-8')
    goal_f.write(xxx)
    goal_f.close()

In [0]:
# Checking out
with open(goal_d + 'Economics.txt') as f:
  t = f.read()
  
print(t[:500])

введение в в российской экономике начался восстановительный рост. однако он остается слабым и неустойчивым при этом сохраняются угрозы новой рецессии статья подготовлена при финансовой поддержке рффи проект санкт петербургский государственный университет вестник санкт петербургского университета. вып. экономика шедших десятилетия россия пережила три циклических спада характерных для среднесрочных колебаний капиталистической экономики. при всех существенных отличиях свойственных этим кризисам в п


In [0]:
# preprocessing


#Склеиваем все тексты в один файл для каждого домена, а потом эти файлы в один общий для всего корпуса.

def make_raws(root=cat_d):
    failed = []
    for d in os.listdir(root):
        with open((os.path.join(root, d) + r'.txt'), 'a', encoding='utf-8') as domain:
            for f in os.listdir(os.path.join(root, d)):
                print(f)
                with open(os.path.join(root, d, f), 'r', encoding='utf-8') as t:
                    try:
                        text = t.read()
                        domain.write(text)
                        domain.write('\n')
                    except UnicodeDecodeError:
                        failed.append(os.path.join(root, d, f))
    print(failed)
    
make_raws()

In [0]:
def make_corpus(root='.'):
    for root, dirs, files in os.walk(root):
        for f in files:
            with open(os.path.join(root, 'CAT.txt'), 'a', encoding='utf8') as corpora:
                with open(os.path.join(root, f), 'r', encoding='utf8', errors='ignore') as f:
                    domain = f.read()
                corpora.write(domain)

In [0]:
make_corpus(root=goal_d)

## Разметка UDpipe
  by A. Klimov. (модель?)
  
  Довольно корявая: личные имена-фамилии очень плохо парсятся, причем нет такого тэга как Proper noun (или есть, но я не нашла). Синтаксис ну очень паршиво работает: много мусора и неправильно разрешенной омонимии.
  
  Саша всё сделал, лежит на гугл-диске в папке новые conll по доменам.
  
  Ниже находится код для работы с гугл-диском. (Ignore it)

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 21.1MB/s 


In [0]:
address = r'https://drive.google.com/drive/folders/164afzH3s2fItPvW8hZzToZo1X3KcHCSC'

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '.txt' and 'root' in parents"}).GetList()
print(listed)

[GoogleDriveFile({'kind': 'drive#file', 'id': '1pgdOd_je-5xe0WvX6kqHAyBcFgw943KK29JB2JRdl3Y', 'etag': '"LIfz66WQFJW5vkUY9zfkRsF1GXw/MTUxNjAzMTcxNDM3OA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1pgdOd_je-5xe0WvX6kqHAyBcFgw943KK29JB2JRdl3Y', 'webContentLink': 'https://drive.google.com/uc?id=1pgdOd_je-5xe0WvX6kqHAyBcFgw943KK29JB2JRdl3Y&export=download', 'alternateLink': 'https://docs.google.com/document/d/1pgdOd_je-5xe0WvX6kqHAyBcFgw943KK29JB2JRdl3Y/edit?usp=drivesdk', 'embedLink': 'https://docs.google.com/document/d/1pgdOd_je-5xe0WvX6kqHAyBcFgw943KK29JB2JRdl3Y/preview?ouid=112440367154326798730', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/vnd.google-apps.document', 'thumbnailLink': 'https://docs.google.com/feeds/vt?gd=true&id=1pgdOd_je-5xe0WvX6kqHAyBcFgw943KK29JB2JRdl3Y&v=1&s=AMedNnoAAAAAXI9-a-fRBOktJUY8bC4KiTc7dlRfj1bM&sz=s220', 'title': 'группы.txt', 'mimeType': 'application/vnd.google-apps.document', 'labels': {'starred': False, 'hi

### Полуинтерактивная проверка корпуса


## Извлечение коллокаций

Основано на https://github.com/MariaFjodorowa/catandthekittens/tree/develop/collocations/collocation_frequencies
by Anna Dmitrieva

Код переработан для лучшей читаемости и оформлен по pep8

In [0]:
pip install conllu

In [0]:
# Парсим конллу
from conllu import parse, parse_tree
import collections

In [0]:
def parser(filename):
  with open(filename, 'r', encoding='utf-8') as f:
    data = f.read()
  tree = parse(data)
  for token in tree:
    yield token

In [0]:
import os
os.listdir('/content/gdrive/My Drive/Новые conll по доменам/NewVers')

['CleanedEco.conllu',
 'CleanedLaw.conllu',
 'CleanedLing.conllu',
 'CleanedPsyEdu.conllu',
 'CleanedSocioHist.conllu',
 'CleanedCAT.conllu',
 'Domains_collocations',
 'PsyEdu.xlsx',
 'Eco.conllu.xlsx']

In [0]:
wrongs = 0
with open('/content/gdrive/My Drive/Новые conll по доменам/NewVers/CleanedCAT.conllu', 'r', encoding='utf-8') as f:
  n = 0
  while n < 1001:
    line = f.readline()
    print(line)
    answer = input('Норм? 0 - да, 1 - нет')
    if answer != '':
      wrongs += int(answer)
      n += 1
print('Процент ошибок: ', str(wrongs/1000))

  

    

# newdoc

# newpar

# sent_id = 1

# text = ВведениеВ г. в российской экономике начался восстановительный рост.

1	ВведениеВ	ВведениеВ	VERB	_	Aspect=Perf|Tense=Past|VerbForm=Conv|Voice=Act	6	advcl	_	SpacesAfter=\s 

2	г.	год	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	1	obj	_	_

3	в	в	ADP	_	_	5	case	_	SpacesAfter= 

4	российской	российский	ADJ	_	Case=Loc|Degree=Pos|Gender=Fem|Number=Sing	5	amod	_	_

5	экономике	экономика	NOUN	_	Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing	1	obl	_	_

6	начался	начаться	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Mid	0	root	_	_

7	восстановительный	восстановительный	ADJ	_	Case=Nom|Degree=Pos|Gender=Masc|Number=Sing	8	amod	_	_

8	рост	рост	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	6	nsubj	_	SpaceAfter=No

9	.	.	PUNCT	_	_	6	punct	_	_



# sent_id = 2

# text = Однако он остается слабым и неустойчивым при этом сохраняются угрозы новой рецессии Статья подготовлена при финансовой поддержке РФФИ проект 

In [0]:
# Извлекаем слова
def get_words(tree):
  words = []
  for sentence in tree:
    for token in sentence:
      if token['form'] != '_' and token['upostag'] != '_' and token['upostag']!='NONLEX' and token['form'] not in r'[]\/':
        for wordform in token['form'].lower().split():
          words.append((wordform, token['upostag']))
  return words # arr of tuples

In [0]:
def get_ngrams(words, n):
    # arr of tuples, int
    ngrams = []
    if n == 1:
        for i in words:
            if i[1] != 'SENT':
                ngrams.append(i)
    else:
      
        for i in range(len(words) - n):
            ngram = words[i:(i + n)]
            for tup in ngram:
                if tup[1]=='SENT':
                    ngram.clear()
                    break
            if ngram:
                word = ''
                tag = ''
                for tup in ngram:
                  # хз, насколько осмысленно собирать их в строку с пробелами, а не в массив
                  word = word + tup[0] + ' '
                  tag = tag + tup[1] + ' '
                ngrams.append((word[:-1], tag[:-1]))
    return ngrams #arr of tuples of two strings

In [0]:
words = get_words(tree)
corpus_len = len(words)

[('страницы', 'NOUN'), ('молодых', 'ADJ'), ('ученых', 'ADJ'), ('ю.', 'PROPN'), ('м.', 'PROPN'), ('дегтяревапроблема', 'NOUN'), ('использования', 'NOUN'), ('непрограммной', 'ADJ'), ('музыки', 'NOUN'), ('в', 'ADP'), ('процессе', 'NOUN'), ('обучения', 'NOUN'), ('в', 'ADP'), ('основной', 'ADJ'), ('школестатья', 'NOUN'), ('посвящена', 'VERB'), ('проблеме', 'NOUN'), ('восприятия', 'NOUN'), ('непрограммной', 'ADJ'), ('музыки', 'NOUN'), ('подростками', 'NOUN'), ('в', 'ADP'), ('условиях', 'NOUN'), ('урока', 'NOUN'), ('музыки', 'NOUN'), ('в', 'ADP'), ('общеобразовательной', 'ADJ'), ('школе', 'NOUN'), ('.', 'PUNCT'), ('автор', 'NOUN')]


In [0]:
import os
import pandas as pd
from pandas import ExcelWriter
import datrie
import math

def make_sheet(n, domain, goal):
  names = {1: 'unigrams', 2: 'bigrams', 3: 'trigrams', 4: 'quadrograms', 5: 'fivegrams', 6: 'sixgrams'}
  if n == 1:
    make_unigrams(domain)
  else:
    try:
      n_freq = datrie.Trie.load(names[n])
    except:
      ngrams = get_ngrams(words, n)
      ngrams_counts = collections.Counter(ngrams)
      n_freq = freq_trie(ngrams_counts)
      n_freq.save(domain + names[n])
      n_freq = datrie.Trie.load(names[n])
    n_1freq = datrie.Trie.load(names[n-1])
    df = counts_table(n_freq, n_1freq)
    with ExcelWriter(goal + domain +'.xlsx') as writer:
      df.to_excel(writer, sheet_name=names[n])
path = '/content/gdrive/My Drive/Новые conll по доменам/NewVers'

domains = [i for i in os.listdir(path) if os.path.isfile(path + i)]
goal = path + 'Domains_collocations/'
for domain in domains:
  domain_tree = parser(path + domain)
  words = get_words(domain_tree)
  corpus_len = len(words)
  for i in range (1, 7):
    make_sheet(i, domain, goal)



In [0]:
print(unigrams[:20])
print(bigrams[:20])

[('страницы', 'NOUN'), ('молодых', 'ADJ'), ('ученых', 'ADJ'), ('ю.', 'PROPN'), ('м.', 'PROPN'), ('дегтяревапроблема', 'NOUN'), ('использования', 'NOUN'), ('непрограммной', 'ADJ'), ('музыки', 'NOUN'), ('в', 'ADP'), ('процессе', 'NOUN'), ('обучения', 'NOUN'), ('в', 'ADP'), ('основной', 'ADJ'), ('школестатья', 'NOUN'), ('посвящена', 'VERB'), ('проблеме', 'NOUN'), ('восприятия', 'NOUN'), ('непрограммной', 'ADJ'), ('музыки', 'NOUN')]
[('страницы молодых', 'NOUN ADJ'), ('молодых ученых', 'ADJ ADJ'), ('ученых ю.', 'ADJ PROPN'), ('ю. м.', 'PROPN PROPN'), ('м. дегтяревапроблема', 'PROPN NOUN'), ('дегтяревапроблема использования', 'NOUN NOUN'), ('использования непрограммной', 'NOUN ADJ'), ('непрограммной музыки', 'ADJ NOUN'), ('музыки в', 'NOUN ADP'), ('в процессе', 'ADP NOUN'), ('процессе обучения', 'NOUN NOUN'), ('обучения в', 'NOUN ADP'), ('в основной', 'ADP ADJ'), ('основной школестатья', 'ADJ NOUN'), ('школестатья посвящена', 'NOUN VERB'), ('посвящена проблеме', 'VERB NOUN'), ('проблеме вос

In [0]:
# На всякий случай, чтобы ничего не терялось
def save_ngram(ngram, filename):
  ngram_lines = []
  for tup in ngram:
    tupline = tup[0] + '!' + tup[1]
    ngram_lines.append(tupline)
  for line in ngram_lines:
    with open(filename, 'a', encoding='utf-8') as f:
      f.write(line)
  print('Done!')
  
def load_ngram(filename):
  with open(filename, 'r', encoding='utf-8') as f:
    ngramlines = f.readlines()
  ngram = [tupline.split('!') for tupline in ngramlines]
  return ngram 

In [0]:
ngr = [unigrams, bigrams]
for i in ngr:
  save_ngram(i, cat_conllu + 'PsyEdu' + str(i))
  print('Done')

In [0]:
unigram_counts = collections.Counter(unigrams)
bigram_counts = collections.Counter(bigrams)
# trigram_counts = collections.Counter(trigrams)
# quadrogram_counts = collections.Counter(quadrograms)
# fivegram_counts = collections.Counter(fivegrams)
# sixgram_counts = collections.Counter(sixgrams)

In [0]:
pip install datrie

    100% |████████████████████████████████| 194kB 10.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/2b/8b/31/fe6fb3dd47e34a4b8cb7505c501676f962f733f0ed9885ede7
Successfully built datrie


In [0]:
import datrie
import math

In [0]:

small=1e-20
def freq_trie(ngrams):
    trie = datrie.Trie('йцукенгшщзхъфывапролджэячсмитьбюё -,:;/ MNBVCXZLKJHGFDSAPOIUYTREWQ')
    for i in ngrams.items():
        trie[i[0][0] + '/' + i[0][1]] = float(i[1]) + small
    return trie

In [0]:
uni_freq = freq_trie(unigram_counts)
uni_freq.save('unigram')
bi_freq = freq_trie(bigram_counts)
bi_freq.save('bigram')
tri_freq = freq_trie(trigram_counts)
tri_freq.save('trigram')
quadro_freq = freq_trie(quadrogram_counts)
quadro_freq.save('quadrogram')
five_freq = freq_trie(fivegram_counts)
five_freq.save('fivegram')
six_freq = freq_trie(sixgram_counts)
six_freq.save('sixgram')

NameError: ignored

In [0]:
print(bi_freq.items()[1000:1050])

[('- контекстного/PUNCT ADJ', 3.0), ('- контекстногообучения/PUNCT NOUN', 1.0), ('- контекстную/PUNCT ADJ', 1.0), ('- контроль/PUNCT NOUN', 1.0), ('- контур/PUNCT NOUN', 2.0), ('- контуров/PUNCT NOUN', 1.0), ('- контуром/PUNCT NOUN', 1.0), ('- конференции/PUNCT NOUN', 1.0), ('- конфликты/PUNCT NOUN', 1.0), ('- конформизм/PUNCT NOUN', 1.0), ('- конформное/PUNCT ADJ', 1.0), ('- концептуальности/PUNCT NOUN', 1.0), ('- концепции/PUNCT NOUN', 16.0), ('- концепцию/PUNCT NOUN', 2.0), ('- концепция/PUNCT NOUN', 1.0), ('- концепцияобучаются/PUNCT VERB', 1.0), ('- координационный/PUNCT ADJ', 1.0), ('- копии/PUNCT NOUN', 1.0), ('- коричневый/PUNCT ADJ', 1.0), ('- корпоративная/PUNCT ADJ', 2.0), ('- корпорации/PUNCT NOUN', 1.0), ('- коррелирует/PUNCT VERB', 2.0), ('- корреляции/PUNCT NOUN', 1.0), ('- корреляционного/PUNCT ADJ', 2.0), ('- корреляционное/PUNCT ADJ', 1.0), ('- корреляционныедедуктивно-корреляционные/PUNCT ADJ', 1.0), ('- корреляционный/PUNCT ADJ', 9.0), ('- корреляционными/PUNCT ADJ'

In [0]:
def L(k, n, x):
    return (math.pow(x,k) * math.pow((1 - x), (n - k))) + small

In [0]:
# Делаем датасет с коллокациями по разным мерам
# Работает - не трогай!
# А оно и не работает, лол
import pandas as pd
def counts_table(ngram_freq_trie, gram_freq_trie):
    ngrams = []
    tags = []
    freq = []
    raw = []
    lr = []
    pm = []
    ts = []
    for i in ngram_freq_trie.items():
        ngram_words = i[0].split('/')[0]
        ngrams.append(ngram_words) # ngram - массив n-грамм
        ngram_tags = i[0].split('/')[1]
        tags.append(ngram_tags) # массив тэгов
        c12 = i[1]
        freq.append(i[1])
        gram_words = ' '.join(ngram_words.split()[:-1])
        gram_tags = ' '.join(ngram_tags.split()[:-1])
        c1 = small
        try:
            c1 = gram_freq_trie[gram_words + '/' + gram_tags]
        except:
            pass
        last_word = ngram_words.split()[-1]
        last_tag = ngram_tags.split()[-1]
        c2 = small
        try:
            c2 = uni_freq[last_word + '/' + last_tag]
        except:
            pass
        # Частота встречаемости во всем корпусе
        raw_freq = i[1]/corpus_len
        raw.append(raw_freq)
        # t score
        ph0 = (c1 / corpus_len) * (c2 / corpus_len)
        ph1 = c12 / corpus_len
        t_score = (ph1 - ph0) / math.sqrt(ph1 / corpus_len)
        ts.append(t_score)
        
        pp12 = c12 / corpus_len
        pp1 = c1 / corpus_len
        pp2 = c2 / corpus_len
        pmi = math.log(pp12/(pp1*pp2), 2.0)
        pm.append(pmi)
        
        # likelyhood ratio
        p = c2 / corpus_len
        p1 = c12 / c1
        p2 = (c2 - c12) / (corpus_len - c1)
        lik_rat = -2*(math.log(L(c12,c1,p)) + math.log(L(c2 - c12, corpus_len - c1, p)) - math.log(L(c12, c1, p1)) - math.log(L(c2 - c12, corpus_len - c1, p2)))
        lr.append(lik_rat)
    df = pd.DataFrame({'ngram':ngrams, 'tag':tags, 'frequency':freq, 'raw frequency':raw, 'likelihood ratio':lr, 'pmi':pm, 't score':ts})
    return df


In [0]:
def make_unigrams(domain):
  unigrams = get_ngrams(words, 1)
  unigram_counts = collections.Counter(unigrams)
  uni_freq = freq_trie(unigram_counts)
  uni_freq.save('unigrams')
  # uni_freq = datrie.Trie.load('unigram')
  uni = []
  uni_t = []
  uni_f = []
  uni_r = []
  for i in uni_freq.items():
      uni_words = i[0].split('/')[0]
      uni.append(uni_words)
      uni_tags = i[0].split('/')[1]
      uni_t.append(uni_tags)
      uni_f.append(i[1])
      uni_r.append(i[1]/corpus_len)
  uni_df = pd.DataFrame({'ngram':uni, 'tag':uni_t, 'frequency':uni_f, 'raw frequency':uni_r})
  with ExcelWriter('/content/gdrive/My Drive/Новые conll по доменам/NewVers/' + domain +'.xlsx') as writer:
      uni_df.to_excel(writer, sheet_name='unigrams')

In [0]:
import os

def make_sheet(n, domain, goal):
  names = {1: 'unigrams', 2: 'bigrams', 3: 'trigrams', 4: 'quadrograms', 5: 'fivegrams', 6: 'sixgrams'}
  if n == 1:
    make_unigrams(domain)
  else:
    try:
      n_freq = datrie.Trie.load(names[n])
    except:
      ngrams = get_ngrams(words, n)
      ngrams_counts = collections.Counter(ngrams)
      n_freq = freq_trie(ngrams_counts)
      n_freq.save(domain + names[n])
      n_freq = datrie.Trie.load(names[n])
    n_1freq = datrie.Trie.load(names[n-1])
    df = counts_table(n_freq, n_1freq)
    with ExcelWriter(goal + domain +'.xlsx') as writer:
      df.to_excel(writer, sheet_name=names[n])
path = '/content/gdrive/My Drive/Новые conll по доменам/NewVers/'

domains = [i[7:] for i in os.listdir(path) if os.path.isfile(path + i)]
goal = path + 'Domains_collocations/'
for domain in domains:
  for i in range (1, 7):
    make_sheet(i, domain, goal)


NameError: ignored

In [0]:
def make_sheet(n, domain, goal):
  names = {1: 'unigrams', 2: 'bigrams', 3: 'trigrams', 4: 'quadrograms', 5: 'fivegrams', 6: 'sixgrams'}
  if n == 1:
    make_unigrams(domain)
  else:
    try:
      n_freq = datrie.Trie.load(names[n])
    except:
      ngrams = get_ngrams(words, n)
      ngrams_counts = collections.Counter(ngrams)
      n_freq = freq_trie(ngrams_counts)
      n_freq.save(domain + names[n])
      n_freq = datrie.Trie.load(names[n])
    n_1freq = datrie.Trie.load(names[n-1])
    df = counts_table(n_freq, n_1freq)
    with ExcelWriter(goal + domain +'.xlsx') as writer:
      df.to_excel(writer, sheet_name=names[n])

In [0]:
from pandas import ExcelWriter

In [0]:
with ExcelWriter('/content/gdrive/My Drive/Новые conll по доменам/NewVers/PsyEdu.xlsx') as writer:
    counts_table(bi_freq, uni_freq).to_excel(writer, sheet_name='bigrams')

In [0]:
# Засейвить это всё в .xlsx
writer = pd.ExcelWriter('/content/gdrive/My Drive/Новые conll по доменам/NewVers/CleanedPsyEdu.xlsx')
uni_df.to_excel(writer,'unigrams')
counts_table(bi_freq, uni_freq).to_excel(writer,'bigrams')
counts_table(tri_freq, bi_freq).to_excel(writer,'trigrams')
counts_table(quadro_freq, tri_freq).to_excel(writer,'quadrograms')
counts_table(five_freq, quadro_freq).to_excel(writer,'fivegrams')
counts_table(six_freq, five_freq).to_excel(writer,'sixgrams')
writer.save()


In [0]:
from pandas import ExcelWriter

In [0]:

with ExcelWriter('/content/gdrive/My Drive/Новые conll по доменам/NewVers/PsyEdu.xlsx') as writer:
    counts_table(bi_freq, uni_freq).to_excel(writer, sheet_name='bigrams')